# Cart Pole

### 1. Import Libraries

In [3]:
!pip install gym[classic_control]

In [1]:
import os
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

### 2. Load Environment

In [4]:
env = gym.make("CartPole-v1")

In [7]:
episodes = 10

for episode in range(episodes):
    state = env.reset()
    done = False
    reward_cnt = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        new_state, reward, done, info = env.step(action)
        
        reward_cnt += reward
        state = new_state
        
    print(f"Episode: {episode} Score: {reward_cnt}")
    
env.close()

Episode: 0 Score: 38.0
Episode: 1 Score: 18.0
Episode: 2 Score: 17.0
Episode: 3 Score: 13.0
Episode: 4 Score: 14.0
Episode: 5 Score: 10.0
Episode: 6 Score: 39.0
Episode: 7 Score: 16.0
Episode: 8 Score: 28.0
Episode: 9 Score: 28.0


### 3. Train Model

In [8]:
log_path = os.path.join("train", "logs")

**PPO**: Proximal Policy Optimization

In [9]:
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [12]:
model.learn(total_timesteps=10_000)

2022-07-09 14:07:07.901271: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96, got 80
<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 232, got 216
<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


Logging to train/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 487  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 474         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008661149 |
|    clip_fraction        | 0.0976      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00507    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.63        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0144     |
|    value_loss           | 51.7        |
-----------------------------------------
------

In [28]:
episodes = 10

for episode in range(episodes):
    state = env.reset()
    done = False
    reward_cnt = 0
    
    while not done:
        env.render()
        action, _ = model.predict(state)
        new_state, reward, done, info = env.step(action)
        
        reward_cnt += reward
        state = new_state
        
    print(f"Episode: {episode} Score: {reward_cnt}")
    
env.close()

Episode: 0 Score: [158.]
Episode: 1 Score: [177.]
Episode: 2 Score: [208.]
Episode: 3 Score: [181.]
Episode: 4 Score: [313.]
Episode: 5 Score: [387.]
Episode: 6 Score: [235.]
Episode: 7 Score: [231.]
Episode: 8 Score: [333.]
Episode: 9 Score: [189.]


### 4. Change Policy

In [14]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [15]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path, policy_kwargs={"net_arch": net_arch})

Using cuda device


In [16]:
model.learn(total_timesteps=10_000)

Logging to train/logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 611  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 394         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015773349 |
|    clip_fraction        | 0.23        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00107    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.13        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0237     |
|    value_loss           | 18.9        |
-----------------------------------------
------

### 5. Add Callbacks

In [18]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [21]:
save_path = os.path.join("train", "saved_models")

In [20]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=475, verbose=1)

In [22]:
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                            best_model_save_path=save_path,
                            verbose=1)

In [26]:
model = PPO("MlpPolicy", env=env, tensorboard_log=log_path, policy_kwargs={"net_arch": net_arch})

In [27]:
model.learn(total_timesteps=10_000, callback=eval_callback)

/home/prince/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=377.60 +/- 99.67
Episode length: 377.60 +/- 99.67
New best mean reward!


### 6. Save Model

In [29]:
MODEL_PATH = os.path.join("train", "saved_models", "PPO_cart_pole_model")

In [30]:
model.save(MODEL_PATH)

### 7. Delete Model

In [31]:
del model

### 8. Reload Model

In [32]:
model = PPO.load(MODEL_PATH, env=env)

### 9. Evaluate Model

In [33]:
evaluate_policy(model, env=env, n_eval_episodes=10, render=True)

/home/prince/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(450.3, 77.43907282502806)

### 10. View Performance

In [35]:
import tensorboard

In [34]:
training_log_path = os.path.join(log_path, "PPO_3")

In [36]:
!tensorboard --logdir={training_log_path}

2022-07-09 14:25:32.458129: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C
